## Running Locally

### Installing deps

In [ ]:
!pip install torch torchvision numpy matplotlib
!pip install -U git+https://github.com/luca-medeiros/lang-segment-anything.git

### downloading sample image

In [ ]:
!wget https://raw.githubusercontent.com/luca-medeiros/lang-segment-anything/main/assets/car.jpeg

In [ ]:
from PIL import Image
import numpy as np
from lang_sam import LangSAM
from lang_sam.utils import draw_image
import matplotlib.pyplot as plt

# Initialize model
model = LangSAM()

# Open and convert image
image_pil = Image.open('/car.jpeg').convert("RGB")
image_np = np.array(image_pil)

# Text prompt
text_prompt = 'car'

# Predict
masks, boxes, labels, logits = model.predict(image_pil, text_prompt)

# Draw image
image = draw_image(image_np, masks, boxes, labels)

# Plot and save image
plt.imshow(image)
plt.axis('off')  # to hide axes
plt.savefig('car-segmented.jpg', bbox_inches='tight', pad_inches=0)
plt.close()

### Downloading sample images

In [ ]:
%%bash
mkdir assets
wget https://raw.githubusercontent.com/luca-medeiros/lang-segment-anything/main/assets/car.jpeg
wget https://cdn.pixabay.com/photo/2020/06/04/22/43/range-rover-5260745_1280.jpg
wget https://www.financialexpress.com/wp-content/uploads/2020/10/2020-Mercedes-Benz-EQC-Electric-SUV-Launched-Live-Updates-5.jpg
wget https://upload.wikimedia.org/wikipedia/commons/f/f5/Melbourne_Tram_E-Class_6007.jpg
wget https://upload.wikimedia.org/wikipedia/commons/9/9a/Soyuz_TMA-9_launch.jpg
mv *.jpg *.jpeg ./assets

### Modifying the script for a folder

In [ ]:
%timeit
from PIL import Image
import numpy as np
import os
from lang_sam import LangSAM
from lang_sam.utils import draw_image
import matplotlib.pyplot as plt

# Initialize model
model = LangSAM()

# Directory containing images
directory = './assets'

# Output directory
output_directory = './outputs'

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Text prompt to match
text_prompt = 'wheels'

# List all files in directory
for filename in os.listdir(directory):
    # Check if file is an image
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        # Open image
        image_pil = Image.open(os.path.join(directory, filename)).convert("RGB")
        image_np = np.array(image_pil)

        # Predict
        masks, boxes, labels, logits = model.predict(image_pil, text_prompt)

        # If the prompt matches labels in the image
        if text_prompt in labels:
            # Draw image
            image = draw_image(image_np, masks, boxes, labels)

            # Save image
            plt.imshow(image)
            plt.axis('off')  # to hide axes
            plt.savefig(os.path.join(output_directory, filename), bbox_inches='tight', pad_inches=0)
            plt.close()
        else:
            print(f"No '{text_prompt}' found in {filename}")

    else:
        print(f"{filename} is not an image file")


### Building the container

#### Requirements
- NVIDIA GPU
- NVIDIA Container runtime

In [ ]:
!docker build -t expanso/sam:new .

#### Pushing to dockerhub

In [ ]:
!docker push  expanso/sam:new

### Running the job on bacalhau

In [ ]:
%%bash
bacalhau docker run \
--gpu 1 \
-i src=s3://sam-coco/micro/*,opt=region=eu-north-1 \
expanso/sam:new 
-- /bin/bash -c 'python /sam.py --input "/inputs" --output "/outputs" --prompt "person"'